Comment: it might be easier to do all of the arithmetic in a *single* cyclotomic field. We need roots of unity of order 3, 5, 7, and their negations, and one more square root (for `splitT`).
```
N = 2^2*3*5*7
K.<z0> = CyclotomicField(N)
z3 = z0^(N//3)
z5 = z0^(N//5)
z7 = z0^(N//7)
```

In [183]:
# set up relations

N = 2^2*3*5*7
K.<z0> = CyclotomicField(N)
z3 = z0^(N//3)
z6 = z0^(N//6)
z5 = z0^(N//5)
z7 = z0^(N//7)

# (R_5 : R_3), w = 6, 1 total
R5_R3 = [z5, z5^2, z5^3, z5^4, z6^(- 1), z6]
# (R_5 : 2R_3), w = 7, 2 total 
R5_2R3_a = [z5^2, z5^3, z5^4, z6^(- 1), z6, z6^(- 1)*z5, z6*z5]
R5_2R3_b = [z5, z5^3, z5^4, z6^(- 1), z6, z6^(- 1)*z5^2, z6*z5^2]
# (R_5 : 3R_3), w = 8, 2 total
R5_3R3_a = [z5^3, z5^4, z6^(- 1), z6, z6^(- 1)*z5, z6*z5, z6^(- 1)*z5^2, z6*z5^2]
R5_3R3_b = [z5, z5^4, z6^(- 1), z6, z6^(- 1)*z5^2, z6*z5^2, z6^(- 1)*z5^3, z6*z5^3]
# (R_7 : R_3), w = 8, 1 total
R7_R3 = [z7, z7^2, z7^3, z7^4, z7^5, z7^6, z6^(- 1), z6]

list_8 = [R5_R3, R5_2R3_a, R5_2R3_b, R5_3R3_a, R5_3R3_b, R7_R3]
# list_8 = [R5_R3]

In [184]:
def splitT(T, m1, m2, neg=False): # takes in list T of terms without z (relation), indices of two terms m1 < m2 to use to compute z
    z = sqrt(T[m1]^(- 1)*T[m2]^(- 1))
    if neg:
        z = - z
    T1, T2 = findT1T2(T, z) # K: return also the complementary set T2 for later use, see below
    t1 = sum(T1)
    return z, t1, T1, T2

Comment: given $T$ and $z$, $T_1$ can be defined as $zT \cap \overline{zT}$, so the following would work:

In [185]:
def findT1T2(T, z):
    zT = [z * t for t in T]
    T1 = [i for i in zT if ~i in zT]
    T2 = [i for i in zT if i not in T1]
    return T1, T2

In [186]:
def findt1reps(relations): # relations : list of relations
    Tlist = [] # list containing [T, z, t1a, t2a, T1a, T2a] for each T
    t1list = [] # list containing just t1 for each T
    for T in relations:
        for j in range(0, len(T)):
            for k in range(j, len(T)): # only care about the unordered pair {j,k}
                za, t1a, T1a, T2a = splitT(T, j, k)
                Tlist.append([T, za, t1a, T1a, T2a])
                t1list.append(t1a)
                zb, t1b, T1b, T2b = splitT(T, j, k, neg = True)
                Tlist.append([T, zb, t1b, T1b, T2b])
                t1list.append(t1b)
    repeats = {} # dictionary containing any sum t1 that repeats, along with the associated T, z, t1a, t2a, T1a, T2a that lead to this t1
    for l in range(0, len(t1list)):
        t1l = t1list[l]
        if t1l in repeats:
            repeats[t1l].append(Tlist[l])
        else:
            repeats[t1l] = [Tlist[l]]

    # K: Remove duplicates where the same T1 shows up in a different order.
    for i in repeats:
        tmp = []
        for j in range(len(repeats[i])):
            for k in range(j+1, len(repeats[i])):
                if repeats[i][j][0] == repeats[i][k][0] and \
                    repeats[i][j][2] == repeats[i][k][2] and \
                    set(repeats[i][j][3]) == set(repeats[i][k][3]):
                    break
            else:
                tmp.append(repeats[i][j])
        repeats[i] = tmp

# K: ignore t=0, this only happens if T1 = T.
    del repeats[0]

    return repeats

Comment: the following function finds a relation of minimal length among a list of roots of unity. This will be needed for postprocessing the previous results.

In [187]:
import itertools

In [188]:
def minimal_relation(l):
    for i in range(1, len(l)+1):
        for s in itertools.combinations(l, i):
            if sum(s) == 0:
                return(s)
    raise ValueError("No relation found")


Similar but now we look for a symmetric (conjugation-stable) relation.

In [189]:
def minimal_symmetric_relation(l):
    for i in range(1, len(l)+1):
        for s in itertools.combinations(l, i):
            if sum(s) == 0 and all(~i in s for i in s):
                return(s)
    raise ValueError("No symmetric relation found")


In [190]:
d = findt1reps(list_8)
for i in d.keys():
    if len(d[i]) > 1:
        for t1, t2 in itertools.combinations(d[i], 2):
            T1a = t1[3]
            T2a = t1[4]
            T1b = t2[3]
            T2b = t2[4]
            for s1 in [-1,1]:
                l = T1a + T1b + [i^s1 for i in T2a] + T2b
                t = minimal_relation(l)
                assert not (all(~i in l for i in l) and any(~i in t for i in t) and not all(~i in t for i in t))
print('done, no errors')

done, no errors
